Lambda School Data Science, Unit 2: Predictive Modeling

# Kaggle Challenge, Module 3


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.

## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### More Categorical Encodings

**1.** The article **[Categorical Features and Encoding in Decision Trees](https://medium.com/data-design/visiting-categorical-features-and-encoding-in-decision-trees-53400fa65931)** mentions 4 encodings:

- **"Categorical Encoding":** This means using the raw categorical values as-is, not encoded. Scikit-learn doesn't support this, but some tree algorithm implementations do. For example, [Catboost](https://catboost.ai/), or R's [rpart](https://cran.r-project.org/web/packages/rpart/index.html) package.
- **Numeric Encoding:** Synonymous with Label Encoding, or "Ordinal" Encoding with random order. We can use [category_encoders.OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html).
- **One-Hot Encoding:** We can use [category_encoders.OneHotEncoder](http://contrib.scikit-learn.org/categorical-encoding/onehot.html).
- **Binary Encoding:** We can use [category_encoders.BinaryEncoder](http://contrib.scikit-learn.org/categorical-encoding/binary.html).


**2.** The short video 
**[Coursera — How to Win a Data Science Competition: Learn from Top Kagglers — Concept of mean encoding](https://www.coursera.org/lecture/competitive-data-science/concept-of-mean-encoding-b5Gxv)** introduces an interesting idea: use both X _and_ y to encode categoricals.

Category Encoders has multiple implementations of this general concept:

- [CatBoost Encoder](http://contrib.scikit-learn.org/categorical-encoding/catboost.html)
- [James-Stein Encoder](http://contrib.scikit-learn.org/categorical-encoding/jamesstein.html)
- [Leave One Out](http://contrib.scikit-learn.org/categorical-encoding/leaveoneout.html)
- [M-estimate](http://contrib.scikit-learn.org/categorical-encoding/mestimate.html)
- [Target Encoder](http://contrib.scikit-learn.org/categorical-encoding/targetencoder.html)
- [Weight of Evidence](http://contrib.scikit-learn.org/categorical-encoding/woe.html)

Category Encoder's mean encoding implementations work for regression problems or binary classification problems. 

For multi-class classification problems, you will need to temporarily reformulate it as binary classification. For example:

```python
encoder = ce.TargetEncoder(min_samples_leaf=..., smoothing=...) # Both parameters > 1 to avoid overfitting
X_train_encoded = encoder.fit_transform(X_train, y_train=='functional')
X_val_encoded = encoder.transform(X_train, y_val=='functional')
```

**3.** The **[dirty_cat](https://dirty-cat.github.io/stable/)** library has a Target Encoder implementation that works with multi-class classification.

```python
 dirty_cat.TargetEncoder(clf_type='multiclass-clf')
```
It also implements an interesting idea called ["Similarity Encoder" for dirty categories](https://www.slideshare.net/GaelVaroquaux/machine-learning-on-non-curated-data-154905090).

However, it seems like dirty_cat doesn't handle missing values or unknown categories as well as category_encoders does. And you may need to use it with one column at a time, instead of with your whole dataframe.

**4. [Embeddings](https://www.kaggle.com/learn/embeddings)** can work well with sparse / high cardinality categoricals.

_**I hope it’s not too frustrating or confusing that there’s not one “canonical” way to encode categorcals. It’s an active area of research and experimentation! Maybe you can make your own contributions!**_

### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [30]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module3')

In [31]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv('https://raw.githubusercontent.com/davidanagy/DS-Unit-2-Kaggle-Challenge/master/data/waterpumps/train_features.csv'), 
                 pd.read_csv('https://raw.githubusercontent.com/davidanagy/DS-Unit-2-Kaggle-Challenge/master/data/waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv('https://raw.githubusercontent.com/davidanagy/DS-Unit-2-Kaggle-Challenge/master/data/waterpumps/test_features.csv')
sample_submission = pd.read_csv('https://github.com/davidanagy/DS-Unit-2-Kaggle-Challenge/raw/master/data/waterpumps/sample_submission.csv')

In [32]:
# Copying my earlier code

import numpy as np

def remove_zeroes(X):
  X = X.copy()
  
  X['latitude'] = X['latitude'].replace(-2e-08, 0)
  
  zeroes = ['gps_height', 'longitude', 'latitude', 'population', 'construction_year']
  for col in zeroes:
    X[col] = X[col].replace(0, np.nan)
  
  return X

def datetime_features(X):
  X = X.copy()
  
  X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
  
  X['year_recorded'] = X['date_recorded'].dt.year
  
  X['construction_year'] = X['construction_year'].fillna(np.around(np.mean(X['construction_year']), decimals=0))
  
  X['time_to_inspection'] = X['year_recorded'] - X['construction_year']
  
  return X

def drop_redundant(X):
  X = X.copy()
  
  redundant_cols = ['recorded_by', 'payment_type', 'region_code', 'date_recorded', 'id']
  
  for col in redundant_cols:
    X = X.drop(col, axis=1)
    
  return X

def wrangle(X):
  X = X.copy()
  
  X = remove_zeroes(X)
  X = datetime_features(X)
  X = drop_redundant(X)
  
  return X

X_train = wrangle(train).drop('status_group', axis=1)
y_train = train['status_group']
X_test = wrangle(test)

In [33]:
import category_encoders as ce
from sklearn.feature_selection import f_classif, SelectKBest
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    StandardScaler(),
    SelectKBest(f_classif),
    LogisticRegression()
)

param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'],
    'selectkbest__k': range(1, len(X_train.columns)+1),
    'logisticregression__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=100,
    cv=5,
    scoring='accuracy',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    9.7s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   11.6s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   25.5s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   37.3s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   49.9s
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:   59.0s
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  89 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 121 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 157 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 197 tasks      | elapsed:  

In [34]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation accuracy', search.best_score_)

Best hyperparameters {'simpleimputer__strategy': 'median', 'selectkbest__k': 36, 'logisticregression__solver': 'lbfgs'}
Cross-validation accuracy 0.6526936026936027


In [35]:
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint, uniform

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=99)
)

param_distributions = {
    'simpleimputer__strategy': ['mean', 'median', 'most_frequent'],
    'randomforestclassifier__n_estimators': randint(50, 500),
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None],
    'randomforestclassifier__max_features': uniform(0, 1),
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=10,
    cv=3,
    scoring='accuracy',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   15.3s
[Parallel(n_jobs=-1)]: Done  11 out of  30 | elapsed:  1.4min remaining:  2.3min
[Parallel(n_jobs=-1)]: Done  15 out of  30 | elapsed:  1.5min remaining:  1.5min
[Parallel(n_jobs=-1)]: Done  19 out of  30 | elapsed:  1.8min remaining:  1.0min
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:  2.3min remaining:   41.4s
[Parallel(n_jobs=-1)]: Done  27 out of  30 | elapsed:  2.5min remaining:   16.3s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  5.1min finished


In [36]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation accuracy', search.best_score_)

Best hyperparameters {'randomforestclassifier__max_depth': 20, 'randomforestclassifier__max_features': 0.7015386858820483, 'randomforestclassifier__n_estimators': 206, 'simpleimputer__strategy': 'mean'}
Cross-validation accuracy 0.8016161616161617


In [37]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_randomforestclassifier__max_depth,param_randomforestclassifier__max_features,param_randomforestclassifier__n_estimators,param_simpleimputer__strategy,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
5,114.813420,1.860060,1.232551,0.032993,20,0.701539,206,mean,"{'randomforestclassifier__max_depth': 20, 'ran...",0.801970,0.800960,0.801919,0.801616,0.000465,1,0.967879,0.967803,0.970000,0.968561,0.001018
8,211.991711,1.185461,1.900429,0.033419,15,0.885041,491,median,"{'randomforestclassifier__max_depth': 15, 'ran...",0.796919,0.794545,0.796061,0.795842,0.000981,2,0.886843,0.893965,0.895960,0.892256,0.003913
1,78.464043,2.537871,1.090128,0.017130,None,0.91037,96,mean,"{'randomforestclassifier__max_depth': None, 'r...",0.795202,0.795657,0.794091,0.794983,0.000658,3,0.999949,0.999975,0.999949,0.999958,0.000012
9,43.800490,0.203105,0.606997,0.027451,15,0.840137,76,mean,"{'randomforestclassifier__max_depth': 15, 'ran...",0.794798,0.792424,0.796313,0.794512,0.001600,4,0.886540,0.895152,0.894066,0.891919,0.003829
2,14.954711,0.670682,2.161393,0.275787,15,0.0530104,306,median,"{'randomforestclassifier__max_depth': 15, 'ran...",0.777121,0.777020,0.776414,0.776852,0.000312,5,0.831616,0.842146,0.843535,0.839099,0.005322
4,38.320987,1.619583,0.687519,0.068054,10,0.922862,78,most_frequent,"{'randomforestclassifier__max_depth': 10, 'ran...",0.760707,0.762020,0.761768,0.761498,0.000569,6,0.787197,0.792601,0.789924,0.789907,0.002206
6,31.383370,0.242379,0.926531,0.066646,5,0.401145,268,median,"{'randomforestclassifier__max_depth': 5, 'rand...",0.720354,0.719293,0.723182,0.720943,0.001641,7,0.724369,0.722475,0.721843,0.722896,0.001073
7,30.142618,0.187024,0.658657,0.041114,5,0.740133,141,mean,"{'randomforestclassifier__max_depth': 5, 'rand...",0.722222,0.717020,0.722121,0.720455,0.002429,8,0.727323,0.721035,0.719722,0.722694,0.003317
3,71.554897,2.503627,1.155833,0.016271,5,0.689805,360,most_frequent,"{'randomforestclassifier__max_depth': 5, 'rand...",0.722828,0.716212,0.720606,0.719882,0.002749,9,0.727146,0.719646,0.717879,0.721557,0.004018
0,11.335455,0.140043,0.485197,0.026296,5,0.556914,60,mean,"{'randomforestclassifier__max_depth': 5, 'rand...",0.720859,0.716162,0.722222,0.719747,0.002596,10,0.723838,0.720707,0.719343,0.721296,0.001882


In [22]:
pipeline = search.best_estimator_

y_pred = pd.DataFrame(pipeline.predict(X_test), columns=['status_group'])

submission1 = pd.concat([test['id'], y_pred], axis=1)

submission1.to_csv('water-submission-12.csv', index=None, header=True)

In [28]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
    ('encoder', ce.BinaryEncoder()),
    ('imputer', SimpleImputer()),
    ('classifier', RandomForestClassifier())
])

param_grid = {
    'encoder': [ce.BinaryEncoder(), ce.OrdinalEncoder()],
    'imputer__strategy': ['mean', 'median', 'most_frequent'],
    'classifier__n_estimators': [50, 100, 150, 200, 300],
    'classifier__max_depth': [5, 10, 15, 20, None],
    'classifier__min_samples_leaf': [1, 5, 10, 15, 20, 30, 50, 100],
    'classifier__max_features': [0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.85, None]
}

grid = GridSearchCV(pipeline, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)

grid.fit(X_train, y_train);

# This was taking forever so I interrupted it.

KeyboardInterrupt: 

In [38]:
pipeline = make_pipeline(
    ce.BinaryEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=99)
)

param_distributions = {
    'simpleimputer__strategy': ['mean', 'median', 'most_frequent'],
    'randomforestclassifier__n_estimators': randint(50, 500),
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None],
    'randomforestclassifier__max_features': uniform(0, 1),
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=10,
    cv=3,
    scoring='accuracy',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  11 out of  30 | elapsed:  5.4min remaining:  9.3min
[Parallel(n_jobs=-1)]: Done  15 out of  30 | elapsed:  7.4min remaining:  7.4min
[Parallel(n_jobs=-1)]: Done  19 out of  30 | elapsed:  9.9min remaining:  5.7min
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed: 12.2min remaining:  3.7min
[Parallel(n_jobs=-1)]: Done  27 out of  30 | elapsed: 13.0min remaining:  1.4min
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 14.5min finished


In [39]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation accuracy', search.best_score_)

Best hyperparameters {'randomforestclassifier__max_depth': None, 'randomforestclassifier__max_features': 0.22217915438926905, 'randomforestclassifier__n_estimators': 413, 'simpleimputer__strategy': 'most_frequent'}
Cross-validation accuracy 0.8072558922558922


In [40]:
pipeline = search.best_estimator_

y_pred = pd.DataFrame(pipeline.predict(X_test), columns=['status_group'])

submission2 = pd.concat([test['id'], y_pred], axis=1)

submission2.to_csv('water-submission-13.csv', index=None, header=True)